In [1]:
import full_pipeline as fp
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
path2input_video = "project_video.mp4"

name, extension = path2input_video.split(".")
path2output_video = name + "_processed9" + "." + extension

In [3]:
camera_calibration, perspective_matrix, meters_per_pix = fp.load_parameters()

In [4]:
birds_eye_buffer = []
ind = 0
buffer_size = 4 # 20 if using AND when combining color and gradient thresholds in combine_color_gradient_postpro()

In [5]:
def decorated_pipeline(image):
    global birds_eye_buffer, ind, buffer_size 
    
    result, previous_birds_eye = fp.pipeline(image, camera_calibration, perspective_matrix, \
                                          meters_per_pix, birds_eye_buffer, ret_control_panel=True)
    
    if len(birds_eye_buffer) < buffer_size:
        birds_eye_buffer.append(previous_birds_eye)
    else:
        birds_eye_buffer[ind] = previous_birds_eye
        
    ind = (ind + 1) % buffer_size

    return result

In [6]:
clip = VideoFileClip(path2input_video)
processed = clip.fl_image(decorated_pipeline) # It is lazy-evaluated. It does not do anything until write_videofile is called
processed.write_videofile(path2output_video, audio=False, threads=2)

[MoviePy] >>>> Building video project_video_processed9_bis.mp4
[MoviePy] Writing video project_video_processed9_bis.mp4


100%|█████████▉| 1260/1261 [09:54<00:00,  2.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_processed9_bis.mp4 



In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(path2output_video))